# 학습 결과물 시각화

## mmdetection

In [ ]:
import json

def process_json_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    # 각 문장의 마지막에 쉼표 추가 (단, 마지막 줄은 제외)
    for i in range(len(lines)-1):
        lines[i] = lines[i].rstrip() + ','

    # 마지막 줄에는 쉼표 추가하지 않음
    lines[-1] = lines[-1].rstrip()

    # 전체 내용을 대괄호로 감싸기
    lines = ['['] + lines + [']']

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines('\n'.join(lines))

input_file = "model/learning_result/dino_result.json"
output_file = "model/learning_result/valid_dino_result.json"

process_json_file(input_file, output_file)
print(f"File '{output_file}' created successfully.")

In [ ]:
import json

# JSON 파일 읽기
with open(output_file, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Initialize dictionaries for loss and bbox data
loss_data = {'epoch': [], 'train/loss': [], 'train/cls_loss': [], 'train/box_loss': []}
map_data = {'epoch': [], 'metrics/mAP50-95(B)': [], 'metrics/mAP50(B)': [], 'metrics/mAP75(B)': []}

for entry in json_data:
    # Handling 'loss' keys
    if 'loss' in entry:
        loss_data['train/loss'].append(entry.get('loss', 0))
        loss_data['train/cls_loss'].append(entry.get('loss_cls', 0))
        loss_data['train/box_loss'].append(entry.get('loss_bbox', 0))
        loss_data['epoch'].append(entry.get('epoch', 0))

    # Handling 'coco/bbox_mAP' keys
    if 'coco/bbox_mAP' in entry:
        map_data['metrics/mAP50-95(B)'].append(entry.get('coco/bbox_mAP_50', 0))
        map_data['metrics/mAP50(B)'].append(entry.get('coco/bbox_mAP', 0))
        map_data['metrics/mAP75(B)'].append(entry.get('coco/bbox_mAP_75', 0))
        map_data['epoch'].append(entry.get('step', 0))

In [ ]:
total_count = len(set(loss_data['epoch']))
steps = len(loss_data['epoch'])

epochs_per_step = steps//total_count

In [ ]:
# 새로운 딕셔너리 초기화
averaged_loss_data = {'epoch': [], 'train/loss': [], 'train/cls_loss': [], 'train/box_loss': []}

# loss_data를 epochs_per_step 단위로 묶어 평균 계산
for i in range(0, len(loss_data['epoch']), epochs_per_step):
    epoch_slice = slice(i, i + epochs_per_step)
    
    # 현재 슬라이스에 속하는 데이터의 평균 계산
    averaged_epoch = sum(loss_data['epoch'][epoch_slice]) / len(loss_data['epoch'][epoch_slice])
    averaged_loss = sum(loss_data['train/loss'][epoch_slice]) / len(loss_data['train/loss'][epoch_slice])
    averaged_loss_cls = sum(loss_data['train/cls_loss'][epoch_slice]) / len(loss_data['train/cls_loss'][epoch_slice])
    averaged_loss_bbox = sum(loss_data['train/box_loss'][epoch_slice]) / len(loss_data['train/box_loss'][epoch_slice])
    
    # 평균 값을 새로운 딕셔너리에 추가
    averaged_loss_data['epoch'].append(int(averaged_epoch))
    averaged_loss_data['train/loss'].append(averaged_loss)
    averaged_loss_data['train/cls_loss'].append(averaged_loss_cls)
    averaged_loss_data['train/box_loss'].append(averaged_loss_bbox)

In [ ]:
import matplotlib.pyplot as plt

metrics = ['train/loss', 'train/cls_loss', 'train/box_loss', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/mAP75(B)']

# 2x3 subplot 생성
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Dino learning result by Metrics over Epochs')

# 각 subplot에 선 그래프 그리기
for i, metric in enumerate(metrics):
    row = i // 3
    col = i % 3
    
    if 'loss' in metric: 
        loss_values = averaged_loss_data[metric]
        axs[row, col].plot(averaged_loss_data['epoch'], loss_values, label=metric)
        
        # Annotate with blue circle at epoch 79
        epoch_79_value = loss_values[78]  # 79th epoch is at index 78
        axs[row, col].plot(79, epoch_79_value, 'bo')
        axs[row, col].text(79, epoch_79_value + 0.1, f'Epoch 79: {epoch_79_value:.4f}', color='blue', fontsize=10, ha='center')

    else:
        map_values = map_data[metric]
        axs[row, col].plot(map_data['epoch'], map_values, label=metric)
        
        # Annotate with red circle at epoch 79
        epoch_79_value = map_values[78]  # 79th epoch is at index 78
        axs[row, col].plot(79, epoch_79_value, 'ro')
        axs[row, col].text(79, epoch_79_value - 0.1, f'Epoch 79: {epoch_79_value:.4f}', color='red', fontsize=10, ha='center')

    axs[row, col].set_title(metric)
    axs[row, col].set_xlabel('Epoch')
    axs[row, col].set_ylabel('Value')
    axs[row, col].grid(True)
    axs[row, col].legend()

# 레이아웃 조정
plt.tight_layout(rect=[0, 0, 1, 0.96])

# 그래프를 이미지 파일로 저장
plt.savefig('model/learning_result/dino_learing_visualization.png', facecolor='white')

# 그래프 표시
plt.show()

## YOLO

In [ ]:
import pandas as pd
yolov8_data = pd.read_csv('model/learning_result/yolov8_result.csv')

In [ ]:
yolov8_data.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 열 이름에서 공백 제거
yolov8_data.columns = yolov8_data.columns.str.strip()

# 필요한 열만 추출
selected_columns = ['epoch',  'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'fitness']
yolov8_data_selected = yolov8_data[selected_columns]

# 2x3 subplot 생성
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle(' learning result by Metrics over Epochs')

# 각 subplot에 선 그래프 그리기
for i, metric in enumerate(yolov8_data_selected.columns[1:]):
    row = i // 3
    col = i % 3
    axs[row, col].plot(yolov8_data_selected['epoch'], yolov8_data_selected[metric], label=metric)
    axs[row, col].set_title(metric)
    axs[row, col].set_xlabel('Epoch')
    axs[row, col].set_ylabel('value')
    axs[row, col].grid(True)
    axs[row, col].legend()

    # Annotate values at epoch 200
    value_at_epoch_200 = yolov8_data_selected.loc[yolov8_data_selected['epoch'] == 200, metric].values[0]

    # Color coding based on metric type
    if 'loss' in metric.lower():
        axs[row, col].plot(200, value_at_epoch_200, 'bo')  # Blue circle for epoch 200 value for loss
        axs[row, col].annotate(f'Epoch 200: {value_at_epoch_200:.4f}',
                              xy=(200, value_at_epoch_200), xytext=(200, value_at_epoch_200 + 0.1),
                              color='blue', ha='center', va='bottom',
                              bbox=dict(boxstyle='round', edgecolor='none', facecolor='none'))
    else:
        axs[row, col].plot(200, value_at_epoch_200, 'ro')  # Red circle for epoch 200 value for map metrics
        axs[row, col].annotate(f'Epoch 200: {value_at_epoch_200:.4f}',
                              xy=(200, value_at_epoch_200), xytext=(200, value_at_epoch_200 - 0.1),
                              color='red', ha='center', va='bottom',
                              bbox=dict(boxstyle='round', edgecolor='none', facecolor='none'))

# 레이아웃 조정
plt.tight_layout(rect=[0, 0, 1, 0.96])

# 그래프를 이미지 파일로 저장
plt.savefig('model/learning_result/yolov8_learing_visualization.png', facecolor='white')

# 그래프 표시
plt.show()